In [2]:
import numpy as np
import pandas as pd
import math

In [3]:
def q(x):
    fuel_mass = sum(x)
    mass = 200 + fuel_mass
    height = 200
    velocity = 0
    acceleration = 0
    profit = -fuel_mass
    for step in x:
        if step == 1:
            mass -= 1
            acceleration_up = 45 / mass
            acceleration = acceleration_up - 0.09
            velocity += acceleration
            height += velocity
        else:
            acceleration = -0.09
            velocity += acceleration
            height += velocity

        if height < 0:
            profit += -1000
            return profit
        elif height < 2 and abs(velocity) < 2:
            profit += 2000
            return profit
    return profit 

In [4]:
def init_population(population_size, genom_size):
    # np.random.seed(10)
    population = [np.random.randint(2, size=genom_size) for i in range(population_size)]
    return population

In [5]:
class Solver:
    def __init__(self, t_max, pc, pm, mi):
        self.t_max = t_max  # liczba iteracji
        self.pc = pc        # prawdopodobieństwo krzyżowania
        self.pm = pm        # prawdopodobieństwo mutacji 
        self.mi = mi        # liczba osobników 


    def get_parameters(self):
        param_dict = {
            "liczba_iteracji": self.t_max,
            "p_mutacji": self.pm,
            "p_krzyzowania": self.pc,
            "liczba_osobnikow": self.mi,
        }
        return param_dict


    def calculate_fitness(self, problem, population):
        fitness = [problem(element) for element in population]
        return fitness 


    def find_best_one(self, population, fitness):
        max_index = np.argmax(fitness)
        x_best = population[max_index]
        o_best = q(x_best)
        o_best_2 = max(fitness)
        return x_best, o_best


    def roulette_selection(self, population, population_fitness):
        offseted_fitness = [element + 1200 for element in population_fitness]
        fitness_sum = sum(offseted_fitness)
        selection_probabilities = [element_fitness/fitness_sum for element_fitness in offseted_fitness]
        indexes = list(range(len(population)))
        selected_indexes = list(np.random.choice(indexes, self.mi, p=selection_probabilities))
        selected = [population[index] for index in selected_indexes]
        return selected


    def mutation(self, population):
        mutated_population = []
        for genom in population:
            muted_genom = []
            for gen in genom:
                mute = np.random.random() < self.pm
                if mute:
                    gen = 1 - gen
                muted_genom.append(gen)
            mutated_population.append(muted_genom)
        return mutated_population

    def crossing(self, population):
        paired = list(zip(population[::2], population[1::2]))
        crossed_population = []
        for pair in paired:
            cross = np.random.random() < self.pc
            if cross:
                crossing_index = np.random.randint(1,len(pair[1]))
                a = list(pair[0])
                b = list(pair[1])
                c = a[:crossing_index] + b[crossing_index:]
                d = b[:crossing_index] + a[crossing_index:]
                crossed_population.append(c)
                crossed_population.append(d)
            else:
                crossed_population.append(pair[0])
                crossed_population.append(pair[1])
        return crossed_population

    def solve(self, problem, pop0):
        t = 0
        P = pop0
        O = self.calculate_fitness(problem, P)
        x_best, o_best = self.find_best_one(P , O)
        while ( t < self.t_max):
            selected = self.roulette_selection(P, O)
            mutated = self.mutation(selected)
            crossed = self.crossing(mutated)
            O = self.calculate_fitness(problem, crossed)
            x, o = self.find_best_one(crossed, O)
            if o > o_best:
                x_best = x
                o_best = o
            P = crossed
            t += 1
        return [x_best, o_best]
        

In [6]:
def run_sim(problem, t_max, pc, pm, mi):
    data = []
    for i in range(25):
        # print(50 * "#")
        print("Symulacja nr ", i, "dane (",t_max,pc,pm,mi, ")")
        pop = init_population(population_size=mi, genom_size=200)
        solver = Solver(t_max,pc, pm, mi)
        result = solver.solve(q,pop)
        data.append(result[1])

    average = sum(data) / len(data)
    standard_deviation = math.sqrt(sum([(x - average)**2 for x in data]) / len(data))
    minimum = min(data)
    maximum = max(data)

    data_dict = {
        "average": average,
        "standard_dev": standard_deviation,
        "min": minimum,
        "max": maximum
    }
    return(data_dict)

In [49]:
def analize_pm(pm_values,t_max):
    average_list = []
    standard_dev_list = []
    min_list = []
    max_list = []
    t_max_list = []
    for x in pm_values:
        data_dict = run_sim(problem=q, t_max=t_max, pc=0.5, pm=x, mi=50)
        average_list.append(data_dict["average"])
        standard_dev_list.append(data_dict["standard_dev"])
        min_list.append(data_dict["min"])
        max_list.append(data_dict["max"])
        t_max_list.append(t_max)
    
    data_dict = {
        "t_max": t_max_list,
        "pm": pm_values, 
        "średnia": average_list,
        "odchylenie standardowe": standard_dev_list,
        "min": min_list,
        "max": max_list
    }

    df = pd.DataFrame(data_dict)
    df.set_index("pm")
    df.style.highlight_max()
    display(df)
    # df.to_excel(f'D:\Studia\SEM 5\WSI\excel\dane_pm_{t_max}.xlsx', index=False, header=True)
    df.to_excel(f'D:\Studia\SEM 5\WSI\excel\dane_pm_69.xlsx', index=False, header=True)
    

In [48]:
analize_pm(pm_values=[0, 0.001, 0.01, 0.1, 0.3, 0.5, 0.7, 0.9], t_max=500)

Symulacja nr  0 dane ( 500 0.5 0 50 )
Symulacja nr  1 dane ( 500 0.5 0 50 )
Symulacja nr  2 dane ( 500 0.5 0 50 )
Symulacja nr  3 dane ( 500 0.5 0 50 )
Symulacja nr  4 dane ( 500 0.5 0 50 )
Symulacja nr  5 dane ( 500 0.5 0 50 )
Symulacja nr  6 dane ( 500 0.5 0 50 )
Symulacja nr  7 dane ( 500 0.5 0 50 )
Symulacja nr  8 dane ( 500 0.5 0 50 )
Symulacja nr  9 dane ( 500 0.5 0 50 )
Symulacja nr  10 dane ( 500 0.5 0 50 )
Symulacja nr  11 dane ( 500 0.5 0 50 )
Symulacja nr  12 dane ( 500 0.5 0 50 )
Symulacja nr  13 dane ( 500 0.5 0 50 )
Symulacja nr  14 dane ( 500 0.5 0 50 )
Symulacja nr  15 dane ( 500 0.5 0 50 )
Symulacja nr  16 dane ( 500 0.5 0 50 )
Symulacja nr  17 dane ( 500 0.5 0 50 )
Symulacja nr  18 dane ( 500 0.5 0 50 )
Symulacja nr  19 dane ( 500 0.5 0 50 )
Symulacja nr  20 dane ( 500 0.5 0 50 )
Symulacja nr  21 dane ( 500 0.5 0 50 )
Symulacja nr  22 dane ( 500 0.5 0 50 )
Symulacja nr  23 dane ( 500 0.5 0 50 )
Symulacja nr  24 dane ( 500 0.5 0 50 )
Symulacja nr  0 dane ( 500 0.5 0.00

,t_max,pm,średnia,odchylenie standardowe,min,max
0,500,0.000,1916.04,2.521587,1910,1921
1,500,0.001,1918.28,2.088444,1912,1922
2,500,0.010,1923.12,2.178440,1920,1929
3,500,0.100,1923.32,1.489161,1921,1927
4,500,0.300,1922.08,2.189429,1919,1930
5,500,0.500,1922.48,1.676186,1920,1926
6,500,0.700,1922.72,2.182109,1920,1930
7,500,0.900,1922.20,1.442221,1920,1925


In [38]:
def analize_pc(pc_values, t_max):
    average_list = []
    standard_dev_list = []
    min_list = []
    max_list = []
    t_max_list = []
    for x in pc_values:
        data_dict = run_sim(problem=q, t_max=t_max, pc=x, pm=0.05, mi=50)
        average_list.append(data_dict["average"])
        standard_dev_list.append(data_dict["standard_dev"])
        min_list.append(data_dict["min"])
        max_list.append(data_dict["max"])
        t_max_list.append(t_max)
    
    data_dict = {
        "t_max": t_max_list,
        "pc": pc_values, 
        "średnia": average_list,
        "odchylenie standardowe": standard_dev_list,
        "min": min_list,
        "max": max_list
    }

    df = pd.DataFrame(data_dict)
    df.set_index("pc")
    display(df)
    df.to_excel(f'D:\Studia\SEM 5\WSI\excel\dane_pc_{t_max}.xlsx', index=False, header=True)

In [45]:
analize_pc(pc_values=[0, 0.001, 0.01, 0.1, 0.3, 0.5, 0.7, 0.9], t_max=500)

Symulacja nr  0 dane ( 500 0 0.05 50 )
Symulacja nr  1 dane ( 500 0 0.05 50 )
Symulacja nr  2 dane ( 500 0 0.05 50 )
Symulacja nr  3 dane ( 500 0 0.05 50 )
Symulacja nr  4 dane ( 500 0 0.05 50 )
Symulacja nr  5 dane ( 500 0 0.05 50 )
Symulacja nr  6 dane ( 500 0 0.05 50 )
Symulacja nr  7 dane ( 500 0 0.05 50 )
Symulacja nr  8 dane ( 500 0 0.05 50 )
Symulacja nr  9 dane ( 500 0 0.05 50 )
Symulacja nr  10 dane ( 500 0 0.05 50 )
Symulacja nr  11 dane ( 500 0 0.05 50 )
Symulacja nr  12 dane ( 500 0 0.05 50 )
Symulacja nr  13 dane ( 500 0 0.05 50 )
Symulacja nr  14 dane ( 500 0 0.05 50 )
Symulacja nr  15 dane ( 500 0 0.05 50 )
Symulacja nr  16 dane ( 500 0 0.05 50 )
Symulacja nr  17 dane ( 500 0 0.05 50 )
Symulacja nr  18 dane ( 500 0 0.05 50 )
Symulacja nr  19 dane ( 500 0 0.05 50 )
Symulacja nr  20 dane ( 500 0 0.05 50 )
Symulacja nr  21 dane ( 500 0 0.05 50 )
Symulacja nr  22 dane ( 500 0 0.05 50 )
Symulacja nr  23 dane ( 500 0 0.05 50 )
Symulacja nr  24 dane ( 500 0 0.05 50 )
Symulacja 

,t_max,pc,średnia,odchylenie standardowe,min,max
0,500,0.000,1923.76,1.530490,1920,1926
1,500,0.001,1923.92,1.622837,1920,1927
2,500,0.010,1923.76,1.175755,1921,1926
3,500,0.100,1923.36,1.520000,1921,1928
4,500,0.300,1923.36,1.466424,1921,1927
5,500,0.500,1923.24,1.656019,1921,1928
6,500,0.700,1923.08,1.055272,1921,1926
7,500,0.900,1923.32,1.690444,1921,1928


In [58]:
def analize_t_max(t_max_values):
    average_list = []
    standard_dev_list = []
    min_list = []
    max_list = []
    for x in t_max_values:
        data_dict = run_sim(problem=q, t_max=x, pc=0.5, pm=0.1, mi=50)
        average_list.append(data_dict["average"])
        standard_dev_list.append(data_dict["standard_dev"])
        min_list.append(data_dict["min"])
        max_list.append(data_dict["max"])
    
    data_dict = {
        "t_max": t_max_values, 
        "średnia": average_list,
        "odchylenie standardowe": standard_dev_list,
        "min": min_list,
        "max": max_list
    }

    df = pd.DataFrame(data_dict)
    df.set_index("t_max")
    display(df)
    df.to_excel(f'D:\Studia\SEM 5\WSI\excel\dane_t_max_10000.xlsx', index=False, header=True)

In [59]:
analize_t_max(t_max_values=[10000])

Symulacja nr  0 dane ( 10000 0.5 0.1 50 )
Symulacja nr  1 dane ( 10000 0.5 0.1 50 )
Symulacja nr  2 dane ( 10000 0.5 0.1 50 )
Symulacja nr  3 dane ( 10000 0.5 0.1 50 )
Symulacja nr  4 dane ( 10000 0.5 0.1 50 )
Symulacja nr  5 dane ( 10000 0.5 0.1 50 )
Symulacja nr  6 dane ( 10000 0.5 0.1 50 )
Symulacja nr  7 dane ( 10000 0.5 0.1 50 )
Symulacja nr  8 dane ( 10000 0.5 0.1 50 )
Symulacja nr  9 dane ( 10000 0.5 0.1 50 )
Symulacja nr  10 dane ( 10000 0.5 0.1 50 )
Symulacja nr  11 dane ( 10000 0.5 0.1 50 )
Symulacja nr  12 dane ( 10000 0.5 0.1 50 )
Symulacja nr  13 dane ( 10000 0.5 0.1 50 )
Symulacja nr  14 dane ( 10000 0.5 0.1 50 )
Symulacja nr  15 dane ( 10000 0.5 0.1 50 )
Symulacja nr  16 dane ( 10000 0.5 0.1 50 )
Symulacja nr  17 dane ( 10000 0.5 0.1 50 )
Symulacja nr  18 dane ( 10000 0.5 0.1 50 )
Symulacja nr  19 dane ( 10000 0.5 0.1 50 )
Symulacja nr  20 dane ( 10000 0.5 0.1 50 )
Symulacja nr  21 dane ( 10000 0.5 0.1 50 )
Symulacja nr  22 dane ( 10000 0.5 0.1 50 )
Symulacja nr  23 dane

,t_max,średnia,odchylenie standardowe,min,max
0,10000,1926.0,0.938083,1925,1928


In [54]:
analize_t_max(t_max_values=[10,100,500,1000])

Symulacja nr  0 dane ( 10 0.5 0.1 50 )
Symulacja nr  1 dane ( 10 0.5 0.1 50 )
Symulacja nr  2 dane ( 10 0.5 0.1 50 )
Symulacja nr  3 dane ( 10 0.5 0.1 50 )
Symulacja nr  4 dane ( 10 0.5 0.1 50 )
Symulacja nr  5 dane ( 10 0.5 0.1 50 )
Symulacja nr  6 dane ( 10 0.5 0.1 50 )
Symulacja nr  7 dane ( 10 0.5 0.1 50 )
Symulacja nr  8 dane ( 10 0.5 0.1 50 )
Symulacja nr  9 dane ( 10 0.5 0.1 50 )
Symulacja nr  10 dane ( 10 0.5 0.1 50 )
Symulacja nr  11 dane ( 10 0.5 0.1 50 )
Symulacja nr  12 dane ( 10 0.5 0.1 50 )
Symulacja nr  13 dane ( 10 0.5 0.1 50 )
Symulacja nr  14 dane ( 10 0.5 0.1 50 )
Symulacja nr  15 dane ( 10 0.5 0.1 50 )
Symulacja nr  16 dane ( 10 0.5 0.1 50 )
Symulacja nr  17 dane ( 10 0.5 0.1 50 )
Symulacja nr  18 dane ( 10 0.5 0.1 50 )
Symulacja nr  19 dane ( 10 0.5 0.1 50 )
Symulacja nr  20 dane ( 10 0.5 0.1 50 )
Symulacja nr  21 dane ( 10 0.5 0.1 50 )
Symulacja nr  22 dane ( 10 0.5 0.1 50 )
Symulacja nr  23 dane ( 10 0.5 0.1 50 )
Symulacja nr  24 dane ( 10 0.5 0.1 50 )
Symulacja 

,t_max,średnia,odchylenie standardowe,min,max
0,10,1917.24,2.159259,1915,1925
1,100,1921.24,1.726963,1917,1924
2,500,1923.56,1.498800,1922,1928
3,1000,1923.44,1.098362,1921,1925


In [55]:
def analize_mi(mi_values):
    average_list = []
    standard_dev_list = []
    min_list = []
    max_list = []
    for x in mi_values:
        data_dict = run_sim(problem=q, t_max=200, pc=0.5, pm=0.1, mi=x)
        average_list.append(data_dict["average"])
        standard_dev_list.append(data_dict["standard_dev"])
        min_list.append(data_dict["min"])
        max_list.append(data_dict["max"])
    
    data_dict = {
        "mi": mi_values, 
        "średnia": average_list,
        "odchylenie standardowe": standard_dev_list,
        "min": min_list,
        "max": max_list
    }

    df = pd.DataFrame(data_dict)
    df.set_index("mi")
    display(df)
    df.to_excel(f'D:\Studia\SEM 5\WSI\excel\dane_mi.xlsx', index=False, header=True)

In [56]:
analize_mi(mi_values=[10, 50, 100, 200, 500, 1000])

Symulacja nr  0 dane ( 200 0.5 0.1 10 )
Symulacja nr  1 dane ( 200 0.5 0.1 10 )
Symulacja nr  2 dane ( 200 0.5 0.1 10 )
Symulacja nr  3 dane ( 200 0.5 0.1 10 )
Symulacja nr  4 dane ( 200 0.5 0.1 10 )
Symulacja nr  5 dane ( 200 0.5 0.1 10 )
Symulacja nr  6 dane ( 200 0.5 0.1 10 )
Symulacja nr  7 dane ( 200 0.5 0.1 10 )
Symulacja nr  8 dane ( 200 0.5 0.1 10 )
Symulacja nr  9 dane ( 200 0.5 0.1 10 )
Symulacja nr  10 dane ( 200 0.5 0.1 10 )
Symulacja nr  11 dane ( 200 0.5 0.1 10 )
Symulacja nr  12 dane ( 200 0.5 0.1 10 )
Symulacja nr  13 dane ( 200 0.5 0.1 10 )
Symulacja nr  14 dane ( 200 0.5 0.1 10 )
Symulacja nr  15 dane ( 200 0.5 0.1 10 )
Symulacja nr  16 dane ( 200 0.5 0.1 10 )
Symulacja nr  17 dane ( 200 0.5 0.1 10 )
Symulacja nr  18 dane ( 200 0.5 0.1 10 )
Symulacja nr  19 dane ( 200 0.5 0.1 10 )
Symulacja nr  20 dane ( 200 0.5 0.1 10 )
Symulacja nr  21 dane ( 200 0.5 0.1 10 )
Symulacja nr  22 dane ( 200 0.5 0.1 10 )
Symulacja nr  23 dane ( 200 0.5 0.1 10 )
Symulacja nr  24 dane ( 20

,mi,średnia,odchylenie standardowe,min,max
0,10,1919.60,1.649242,1916,1923
1,50,1922.24,1.795104,1920,1927
2,100,1923.08,2.076921,1920,1928
3,200,1924.40,1.442221,1922,1927
4,500,1925.16,1.433318,1923,1928
5,1000,1925.48,1.268700,1923,1930
